# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.tile_sources as gts

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col=0)

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,college,64.8569,-147.8028,270.12,73,100,3.09,US,1697936750
1,qaqortoq,60.7167,-46.0333,273.46,53,0,2.51,GL,1697936829
2,tiksi,71.6872,128.8694,251.12,100,78,0.74,RU,1697936830
3,albany,42.6001,-73.9662,285.66,81,97,0.89,US,1697936830
4,bredasdorp,-34.5322,20.0403,286.89,90,100,5.91,ZA,1697936830


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size="Humidity",
    width=1500,        
    height=800
)

# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 294.15) & (city_data_df['Max Temp'] <= 301.15)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
16,champerico,14.3000,-91.9167,300.71,76,24,1.96,GT,1697936833
18,taroa,8.7058,171.2275,301.05,81,100,7.85,MH,1697936834
22,touros,-5.1989,-35.4608,298.93,80,29,6.00,BR,1697936835
23,oodweyne,9.4092,45.0640,294.87,59,21,1.86,SO,1697936835
28,guerrero negro,27.9769,-114.0611,295.25,78,0,5.49,MX,1697936640


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City','Country','Lng','Lat','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel'] = ''

# Display sample data
hotel_df 


,City,Country,Lng,Lat,Humidity,Hotel
City_ID,,,,,,
16,champerico,GT,-91.9167,14.3000,76,
18,taroa,MH,171.2275,8.7058,81,
22,touros,BR,-35.4608,-5.1989,80,
23,oodweyne,SO,45.0640,9.4092,59,
28,guerrero negro,MX,-114.0611,27.9769,78,
...,...,...,...,...,...,...
581,thohoyandou,ZA,30.4850,-22.9456,30,
585,tarfaya,MA,-12.9218,27.9358,85,
586,aketi,CD,23.7833,2.7388,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
from requests.structures import CaseInsensitiveDict

# API URL
base_url = "https://api.geoapify.com/v2/places"
api_key = geoapify_key


headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

# Iterate 
for index, row in hotel_df.iterrows():
    latitude, longitude = row['Lat'], row['Lng']

    # Set the params as per documentation
    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{longitude},{latitude},10000',
        'limit': 1,
        'apiKey': api_key
    }

    # API request
    resp = requests.get(base_url, params=params, headers=headers)

    # Check the status code
    if resp.status_code == 200:
        data = resp.json()
        
        # Print to check structure
        print(data)
        
        if 'features' in data and len(data['features']) > 0:
           
            hotel_name = data['features'][0]['properties'].get('name', 'Name not found')
            print(f"Hotel Name for index {index}: {hotel_name}")
           
            hotel_df.at[index, "Hotel"] = hotel_name
        else:
            print(f"No hotel found for index {index}")
    else:
        print(f"Error for index {index} with status code {resp.status_code}")

# Display the updated DataFrame
print(hotel_df)

 

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Posada Del Mar', 'country': 'Guatemala', 'country_code': 'gt', 'state': 'Retalhuleu', 'city': 'Champerico', 'village': 'El Palmo', 'street': 'Carretera Retalhuelu - Champerico', 'lon': -91.90181804672535, 'lat': 14.3063766, 'state_code': 'RE', 'formatted': 'Posada Del Mar, Carretera Retalhuelu - Champerico, - Champerico, Guatemala', 'address_line1': 'Posada Del Mar', 'address_line2': 'Carretera Retalhuelu - Champerico, - Champerico, Guatemala', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Posada Del Mar', 'osm_id': 707088782, 'tourism': 'hotel', 'osm_type': 'w'}}, 'place_id': '51a8de639eb8f956c05929b0c5d1dd9c2c40f00102f9018e51252a0000000092030e506f736164612044656c204d6172'}, 'geomet

In [76]:
hotel_df.head()

,City,Country,Lng,Lat,Humidity,Hotel
City_ID,,,,,,
16,champerico,GT,-91.9167,14.3000,76,Posada Del Mar
18,taroa,MH,171.2275,8.7058,81,
22,touros,BR,-35.4608,-5.1989,80,Siri Paraíso Hotel
23,oodweyne,SO,45.0640,9.4092,59,فندق المدينه
28,guerrero negro,MX,-114.0611,27.9769,78,Cabanas Malarrimo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [85]:
# Configure the map plot_1
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    width=1500,
    height=800,
    hover_cols=["Country",'Hotel']
)
# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Country,Hotel)